In [1]:
from datasets import load_dataset

ds = load_dataset("Chiranjeev007/CIFAR-10_Subset")
print(ds)
# DatasetDict({
#   train:      Dataset(num_rows: 5000),
#   validation: Dataset(num_rows: 500),
#   test:       Dataset(num_rows: 1000)
# })

sample = ds["train"][0]
sample["image"]   # PIL Image 32×32 RGB
sample["label"]   # int 0–9


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.29M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1000
    })
})


9

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from datasets import load_dataset
from huggingface_hub import HfApi
import wandb
import os
import copy
from huggingface_hub import login
from google.colab import userdata

# 1. Make sure you have a secret named 'HF_TOKEN' in the Colab sidebar (Key icon)
# 2. Ensure that secret contains a 'WRITE' token from HF settings
try:
    token = userdata.get('HF_TOKEN')
    login(token=token, add_to_git_credential=True)
    print("Login Successful!")
except Exception as e:
    print(f"Login failed: {e}")

# ==========================
# 1. Load Data from Hugging Face
# ==========================
print("Loading dataset from Hugging Face...")
ds = load_dataset("Chiranjeev007/CIFAR-10_Subset")
print(ds)

# ==========================
# 2. Define Transforms
# ==========================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ==========================
# 3. Custom Dataset Class
# ==========================
class HFImageDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample["image"].convert("RGB")
        label = sample["label"]

        if self.transform:
            image = self.transform(image)

        return image, label

# ==========================
# 4. Create DataLoaders
# ==========================
BATCH_SIZE = 32

train_dataset = HFImageDataset(ds["train"], transform=train_transform)
val_dataset = HFImageDataset(ds["validation"], transform=val_transform)
test_dataset = HFImageDataset(ds["test"], transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

# ==========================
# 5. Load Pretrained ResNet-18
# ==========================
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

print(f"Model loaded on: {DEVICE}")

# ==========================
# 6. Loss, Optimizer, Config
# ==========================
EPOCHS = 10
LR = 1e-3

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ==========================
# 7. Initialize WandB
# ==========================
wandb.init(
    project="minor_b23bb1032",
    config={
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LR,
        "model": "resnet18-pretrained",
        "dataset": "Chiranjeev007/CIFAR-10_Subset",
        "optimizer": "Adam",
    }
)

# ==========================
# 8. Training Loop with WandB Logging
# ==========================
best_val_acc = 0.0
best_model_state = None

for epoch in range(EPOCHS):
    # --- Training Phase ---
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total

    # --- Validation Phase ---
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100.0 * val_correct / val_total

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
    })

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # --- Save Best Model ---
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        torch.save(best_model_state, "best_model.pth")
        print(f"  -> Best model saved! (Val Acc: {val_acc:.2f}%)")

print(f"\nTraining Complete! Best Val Accuracy: {best_val_acc:.2f}%")

# ==========================
# 9. Test Evaluation
# ==========================
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100.0 * test_correct / test_total
print(f"Test Accuracy: {test_acc:.2f}%")
wandb.log({"test_accuracy": test_acc})

# ==========================
# 10. Push Best Model to Hugging Face Hub
# ==========================
HF_REPO_ID = "Prakhar54-byte/HF_minor_b23bb1032"

api = HfApi()

# Create repo (if it doesn't exist)
api.create_repo(repo_id=HF_REPO_ID, exist_ok=True)

# Upload the best model file
api.upload_file(
    path_or_fileobj="best_model.pth",
    path_in_repo="best_model.pth",
    repo_id=HF_REPO_ID,
)

print(f"Model pushed to https://huggingface.co/{HF_REPO_ID}")

wandb.finish()

Login failed: Secret HF_TOKEN does not exist.
Loading dataset from Hugging Face...
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1000
    })
})
Train: 5000, Val: 500, Test: 1000


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded on: cuda


epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
train_accuracy,▁▄▅▆▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_accuracy,▁▄▅▃▆▇██▆▇
val_loss,█▃▅▅▃▂▁▁▂▁
epoch,10
test_accuracy,81.2
train_accuracy,85.86
train_loss,0.42046
val_accuracy,77.4


Epoch [1/10] Train Loss: 1.3303 | Train Acc: 54.68% | Val Loss: 1.2587 | Val Acc: 59.40%
  -> Best model saved! (Val Acc: 59.40%)
Epoch [2/10] Train Loss: 0.9918 | Train Acc: 65.42% | Val Loss: 1.1727 | Val Acc: 64.80%
  -> Best model saved! (Val Acc: 64.80%)
Epoch [3/10] Train Loss: 0.7924 | Train Acc: 72.74% | Val Loss: 0.7982 | Val Acc: 72.20%
  -> Best model saved! (Val Acc: 72.20%)
Epoch [4/10] Train Loss: 0.7207 | Train Acc: 75.42% | Val Loss: 0.8122 | Val Acc: 74.80%
  -> Best model saved! (Val Acc: 74.80%)
Epoch [5/10] Train Loss: 0.6296 | Train Acc: 77.92% | Val Loss: 0.8658 | Val Acc: 72.20%
Epoch [6/10] Train Loss: 0.5826 | Train Acc: 80.54% | Val Loss: 0.6989 | Val Acc: 78.00%
  -> Best model saved! (Val Acc: 78.00%)
Epoch [7/10] Train Loss: 0.5344 | Train Acc: 81.84% | Val Loss: 0.7703 | Val Acc: 76.00%
Epoch [8/10] Train Loss: 0.4941 | Train Acc: 82.46% | Val Loss: 0.8956 | Val Acc: 72.60%
Epoch [9/10] Train Loss: 0.4548 | Train Acc: 84.36% | Val Loss: 0.9422 | Val Acc: 7

HfHubHTTPError: Client error '401 Unauthorized' for url 'https://huggingface.co/api/repos/create' (Request ID: Root=1-69998cc9-01b9b6ff24f0f7f95d36189d;fe7cdbe7-062c-4866-ae48-ded96fa6f62a)
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401

Invalid username or password.